# Library

In [154]:
import pandas as pd
import numpy as np
import os
import re
import collections
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords

# Functions

In [2]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name) as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

# Load Data

### Testing

220 clinical cases.

In [3]:
testing_abbr = pd.read_csv("datasets/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')
testing_met = pd.read_csv("datasets/testing_set/clinical_cases.metadata.testing_set.tsv", sep = '\t')
testing_rel = pd.read_csv("datasets/testing_set/clinical_cases.relations.testing_set.tsv", sep = '\t')

In [4]:
testing_met.head()

,# Document_ID,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-01082006000700014-1.txt,1.txt,1130-0108,2006-07-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082007000300006-7.txt,7.txt,1130-0108,2007-03-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1134-80462009000100005-1.txt,1.txt,1134-8046,2009-02-01,Revista de la Sociedad Española del Dolor v.1...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1137-66272014000300015-1.txt,1.txt,1137-6627,2014-12-01,Anales del Sistema Sanitario de Navarra v.37 ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0365-66912004001200011-1.txt,1.txt,0365-6691,2004-12-01,Archivos de la Sociedad Española de Oftalmolog...,http://scielo.isciii.es/scielo.php?script=sci_...


In [5]:
testing_rel = testing_rel.reset_index()

In [6]:
testing_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [7]:
testing_rel = testing_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [8]:
testing_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0211-69952013000500019-1,SHORT_FORM,3739,ARA II,SHORT-LONG,LONG_FORM,3695,3737,antagonista del receptor de angiotesina II
1,S0211-69952013000500019-1,SHORT_FORM,2793,PCR,SHORT-LONG,LONG_FORM,2798,2837,reacción en cadena de enzima polimerasa
2,S0365-66912004000600008-1,SHORT_FORM,406,AV,SHORT-LONG,LONG_FORM,390,404,agudeza visual
3,S0211-69952012000500025-1,SHORT_FORM,945,angio-TAC,SHORT-LONG,LONG_FORM,908,943,angiotomografía computarizada axial
4,S1130-05582017000200122-1,SHORT_FORM,940,RMN,SHORT-LONG,LONG_FORM,910,938,resonancia magnética nuclear


In [9]:
testing_rel.Relation_type.unique()

array(['SHORT-LONG', 'SHORT-NESTED', 'NESTED-LONG'], dtype=object)

In [10]:
testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [11]:
testing_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142010000500014-1,2037,2044,16SrRNA,16s ribosomal rna,16s ribosomal rno
1,S0004-06142010000500014-1,1349,1351,M.,mycobacterium,mycobacterium
2,S0004-06142010000500014-1,1339,1342,PCR,polymerase chain reaction,polymerase chain reaction
3,S0004-06142010000500014-1,611,615,BHCG,beta-human chorionic gonadotropin,beta-humar chorionic gonadotropin
4,S0004-06142010000500014-1,594,597,CEA,carcinoembrionary antigen,carcinoembrionary antiger


In [12]:
testing_raw = read_texts("datasets/testing_set/testing_set.raw_text/")

In [13]:
testing_raw.head()

,nombre,texto
0,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses...
1,S0210-48062007000700015-1,Varón de 72 años con antecedentes personales d...
2,S0212-71992005000500009-1,"Varón de 81 años, con antecedentes de fibrilac..."
3,S0365-66912004001200011-1,Paciente varón de 52 años que acudió a urgenci...
4,S1130-01082009000900015-1,Varón de 54 años con episodios de pancreatitis...


### Backgound

2879 clinical cases. 220 will be use for clinical evaluation

In [14]:
back_met = pd.read_csv("datasets/background_test/clinical_cases.metadata.background_set.tsv", sep = '\t')

In [15]:
back_met = back_met.rename(columns = {'# Document_ID': 'doc_id'})

In [16]:
back_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0325-00752010000100014.txt,1.txt,0325-0075,02/2010,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
1,S0325-00752013000200014.txt,1.txt,0325-0075,04/2013,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
2,S0325-00752011000400017.txt,1.txt,0325-0075,08/2011,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
3,S0325-00752013000600022.txt,1.txt,0325-0075,12/2013,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
4,S0325-00752008000500013.txt,1.txt,0325-0075,10/2008,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...


In [17]:
back_raw = read_texts("datasets/background_test/background_test_set/")

In [18]:
back_raw.head()

,nombre,texto
0,S0212-71992004000800006-1,Mujer de 57 años de edad con antecedentes pers...
1,S0716-10182014000100007-1,Se recibió en el Hospital Veterinario Puente A...
2,S1130-01082009000200016-1,"Mujer de 47 años de edad, sin antecedentes de ..."
3,S0716-10182015000400016-1,"Escolar de 11 años de edad, género femenino, s..."
4,S1130-01082004001200010-2,"Varón de 25 años, remitido a nuestro Servicio ..."


### Development

146 clinical cases

In [19]:
dev_abbr = pd.read_csv("datasets/development_set/clinical_cases.abbreviations.development_set.tsv", sep = '\t')
dev_met = pd.read_csv("datasets/development_set/clinical_cases.metadata.development_set.tsv", sep = '\t')
dev_rel = pd.read_csv("datasets/development_set/clinical_cases.relations.development_set.tsv", sep = '\t')

In [20]:
dev_met = dev_met.rename(columns = {'# Document_ID': 'doc_id'})

In [21]:
dev_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-14732005000300004-1.txt,1.txt,1130-1473,2005-06-01,Neurocirugía v.16 n.3 2005,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082008000200009-1.txt,1.txt,1130-0108,2008-02-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1137-66272012000300021-1.txt,1.txt,1137-6627,2012-12-01,Anales del Sistema Sanitario de Navarra v.35 ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1699-695X2016000200010-1.txt,1.txt,1699-695X,2016-06-01,Revista Clínica de Medicina de Familia v.9 n....,http://scielo.isciii.es/scielo.php?script=sci_...
4,S1130-01082007001100009-1.txt,1.txt,1130-0108,2007-11-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...


In [22]:
dev_rel = dev_rel.reset_index()

In [23]:
dev_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [24]:
dev_rel = dev_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [25]:
dev_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1888-75462015000400006-1,SHORT_FORM,1436,AP,SHORT-LONG,LONG_FORM,1419,1434,anteroposterior
1,S0210-56912009000700006-2,SHORT_FORM,876,angio-RMN,SHORT-LONG,LONG_FORM,858,874,angiorresonancia
2,S0210-56912009000700006-2,SHORT_FORM,819,RMN,SHORT-LONG,LONG_FORM,789,817,resonancia magnética nuclear
3,S1698-44472005000300015-1,SHORT_FORM,209,ATM,SHORT-LONG,LONG_FORM,177,207,articulación témporomandibular
4,S1698-44472005000300015-1,SHORT_FORM,1125,TC,SHORT-LONG,LONG_FORM,1099,1123,tomografía computerizada


In [26]:
dev_abbr = dev_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [27]:
dev_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000300004-1,1216,1218,C3,tercera vértebra cervical,tercero vértebra cervical
1,S1130-14732005000300004-1,717,719,C2,segunda vértebra cervical,segundo vértebra cervical
2,S1130-14732005000300004-1,3191,3193,C3,tercera vértebra cervical,tercero vértebra cervical
3,S1130-14732005000300004-1,2867,2869,C3,tercera vértebra cervical,tercero vértebra cervical
4,S1130-14732005000300004-1,2862,2864,C2,segunda vértebra cervical,segundo vértebra cervical


In [28]:
dev_raw = read_texts("datasets/development_set/development_set.raw_text/")

In [29]:
dev_raw.head()

,nombre,texto
0,S1139-76322017000200010-1,Niña de dos años y diez meses con antecedentes...
1,S0365-66912005001100008-1,Se presenta el caso de un varón de 45 años que...
2,S1130-01082006001000017-1,Mujer de 42 años de edad con antecedentes pers...
3,S0212-71992005001200008-1,Paciente de 57 años con ingresos hospitalarios...
4,S0365-66912007000300010-1,Paciente de 33 años que el 20-08-05 es traslad...


### Trainning

318 clinical cases

In [30]:
train_abbr = pd.read_csv("datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')
train_met = pd.read_csv("datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [31]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [32]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacia...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_...


In [33]:
train_rel = train_rel.reset_index()

In [34]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [35]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [36]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [37]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [38]:
train_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0210-48062004000500008-1,1650,1652,ml,mililitro,mililitro
1,S0210-48062004000500008-1,708,709,l,litro,litro
2,S0210-48062004000500008-1,704,707,mEq,miliequivalente,miliequivalente
3,S0210-48062004000500008-1,677,681,pCO2,presión parcial de co2,presión parcial de co2
4,S0210-48062004000500008-1,2287,2290,HLA,human leucocyte antigen,human leucocyte antiger


In [39]:
train_raw = read_texts("datasets/trainning_set/training_set.raw_text/")

In [40]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [41]:
train_raw.head()

,doc_id,texto
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...


### Sample test

15 clinical cases

In [42]:
sample_abbr = pd.read_csv("datasets/sample_set/clinical_cases.abbreviations.sample_set.tsv", sep = '\t')
sample_met = pd.read_csv("datasets/sample_set/clinical_cases.metadata.sample_set.tsv", sep = '\t')
sample_rel = pd.read_csv("datasets/sample_set/clinical_cases.relations.sample_set.tsv", sep = '\t')

In [43]:
sample_met = sample_met.rename(columns = {'# Document_ID': 'doc_id'})

In [44]:
sample_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0004-06142006000900015-1,1,0004-0614,2006-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S0004-06142006000600015-1,1,0004-0614,2006-08-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0004-06142007000700014-1,1,0004-0614,2007-09-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0004-06142007000900013-1,1,0004-0614,2007-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0004-06142006000200014-1,1,0004-0614,2006-03-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...


In [45]:
sample_rel = sample_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [46]:
sample_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A_EndOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0004-06142006000700014-1,SHORT_FORM,926,929,CEA,SHORT-LONG,LONG_FORM,896,924,Antígeno Carcino Embrionario
1,S0004-06142005001000011-1,SHORT_FORM,1626,1629,ROT,SHORT-LONG,LONG_FORM,1600,1624,reflejos osteotendinosos
2,S0004-06142005001000011-1,SHORT_FORM,1715,1718,RMN,SHORT-LONG,LONG_FORM,1685,1713,resonancia magnética nuclear
3,S0004-06142005001000011-1,SHORT_FORM,1663,1666,RCP,SHORT-LONG,LONG_FORM,1639,1661,reflejo cutaneoplantar
4,S0004-06142005001000011-1,SHORT_FORM,1808,1811,LCR,SHORT-LONG,LONG_FORM,1783,1806,líquido cefalorraquídeo


In [47]:
sample_abbr = sample_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [48]:
sample_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142005001000011-1,1034,1036,Kg,kilogramo,kilogramo
1,S0004-06142005001000011-1,1031,1033,mg,miligramo,miligramo
2,S0004-06142005001000011-1,196,199,IgA,inmunoglobulina a,inmunoglobulina a
3,S0004-06142005001000011-1,2057,2060,LCR,líquido cefalorraquídeo,líquido cefalorraquídeo
4,S0004-06142005001000011-1,1594,1598,EEII,extremidades inferiores,extremidad inferior


In [49]:
sample_raw = read_texts("datasets/sample_set/sample_set.raw_text/")

In [50]:
sample_raw.head()

,nombre,texto
0,S0004-06142006000600014-1,"Paciente varón, de 40 años de edad, con antece..."
1,S0004-06142006000300015-1,Paciente de 50 años con antecedente de litiasi...
2,S0004-06142007000900013-1,Presentamos el caso de un recién nacido de tre...
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
4,S0004-06142006000700013-1,"Paciente varón, de 63 años de edad, mestizo, d..."


# Sub-track 1

## Found abbreviations

### Regex

In [249]:
#patron = r'\(([A-Z]{2,8})\)'
#patron1 = r'\s[A-Z]{1,3}\s'
#patron2 = r'\s[a-z]{1,3}\s'
patron3 = r'[A-Z]{2,8}'
patron4 = r'\s[a-z]{1,2}\s'
patron5 = r'[a-z]+\-[A-Z]{1,8}'
patron6 = r'[a-z]+\/[a-z]+'
patron7 = r'[A-Z]?[a-z]{1,4}[A-Z]+[a-z]*[1-9]*'
patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7,patron8]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7,patron8))

#### Solution 1

In [53]:
# todos = []
# for _,row in train_raw.iterrows():
#     texto = row['texto']
#     id_texto = row['nombre']
#     ab_found= set(generic_re.findall(texto))
#     #print(texto,id_texto, ab_found)
#     for item in ab_found:
#         abr = {}
#         abr = {'id_texto':id_texto,'abre':item }
#         #print(abr)
#         #abr[id_texto] = {item} | abr.get(id_texto) if abr.get(id_texto) else {item}
#         todos.append(abr)

#todos

In [54]:
# df_abr = pd.DataFrame(todos)

In [55]:
# df_abr.head()


In [ ]:
#df_abr = pd.DataFrame(abr.items(), columns=['# Document_ID', 'abbreviation_m'])

#### Solution 2 (best one)

In [210]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

In [250]:
train_raw['abrev'] = train_raw['texto'].map(lambda x: generic_re.findall(x))

In [251]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i.strip() for i in x])

In [252]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i for i in x if i not in swords])

In [253]:
train_raw.head()

,doc_id,texto,abrev
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]"
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[/mm, /mm, mg/dl, mg/dl, mg/dl, mg/dl, LDH, UI..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[]
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg/dl, mg/dl, mg/dl, AST, /l, ALT, /l, GGT..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[]


In [254]:
mine = train_raw.explode('abrev')

In [255]:
mine['abrev2'] = mine['abrev'].apply(lambda x: str(x).split("/"))

In [256]:
mine = mine.explode('abrev2')

In [257]:
mine['abrev2'].replace('', np.nan, inplace=True)

In [258]:
mine = mine.dropna(subset = ['abrev2'])

In [245]:
#mine = mine.drop_duplicates(subset =['doc_id', 'abrev2'])

#### Join with train_abr to see how many abbreviations I have found

In [259]:
matches = train_abbr[['doc_id', 'Abbreviation']].merge(mine, how = 'left', on='doc_id')

In [260]:
matches.head()

,doc_id,Abbreviation,texto,abrev,abrev2
0,S0210-48062004000500008-1,ml,Paciente de 29 años de edad que acude al Servi...,u,u
1,S0210-48062004000500008-1,ml,Paciente de 29 años de edad que acude al Servi...,pCO2,pCO2
2,S0210-48062004000500008-1,ml,Paciente de 29 años de edad que acude al Servi...,mmHg,mmHg
3,S0210-48062004000500008-1,ml,Paciente de 29 años de edad que acude al Servi...,HCO,HCO
4,S0210-48062004000500008-1,ml,Paciente de 29 años de edad que acude al Servi...,mEq,mEq


Check dataframes for one text

In [261]:
train_raw['texto'].iloc[1]

'Se trataba de un varón de 27 años de edad, que había sufrido una neumonía siendo niño y que no era fumador, ni tenía hábito etílico. Estaba diagnosticado de asma bronquial extrínseco y rinoconjuntivitis alérgica con hipersensibilidad a gramíneas y a olivo. Necesitaba de corticoides inhalados de forma esporádica. Tras un esfuerzo físico importante (salir de costalero en una procesión durante unas fiestas locales y haber dormido muy poco) en el verano de 2003 comenzó con dolor en el hemitórax derecho que en ocasiones se le irradiaba al miembro superior derecho y que aumentaba con la inspiración profunda. Posteriormente le apareció fiebre y sensación disneica por lo que acudió a urgencias del hospital. En la exploración clínica del ingreso se le encontró mal estado general y una lesión en fase cicatricial en la pared del abdomen que podía corresponder a un forúnculo. El murmullo vesicular estaba disminuido en la base derecha. Las exploraciones cardiaca y neurológica fueron normales. No h

In [262]:
mine[mine['doc_id'] == 'S0212-71992005000400009-1']

,doc_id,texto,abrev,abrev2
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",/mm,mm
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",/mm,mm
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,mg
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,dl
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,mg
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,dl
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,mg
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,dl
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,mg
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",mg/dl,dl


In [263]:
train_abbr[train_abbr['doc_id'] == 'S0212-71992005000400009-1']

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
3462,S0212-71992005000400009-1,1384,1387,GOT,glutamic oxalic transaminase,glutamic oxalic transaminar
3463,S0212-71992005000400009-1,1398,1401,GPT,glutamate pyruvate transaminase,glutamate pyruvatar transaminar
3464,S0212-71992005000400009-1,1378,1380,UI,unidad internacional,unidad internacional
3465,S0212-71992005000400009-1,1381,1382,l,litro,litro
3466,S0212-71992005000400009-1,1335,1337,dl,decilitro,decilitro
3467,S0212-71992005000400009-1,1369,1372,LDH,lactato-deshidrogenasa,lactato-deshidrogenasa
3468,S0212-71992005000400009-1,1271,1273,dl,decilitro,decilitro
3469,S0212-71992005000400009-1,1268,1270,mg,miligramo,miligramo
3470,S0212-71992005000400009-1,1247,1249,mg,miligramo,miligramo
3471,S0212-71992005000400009-1,1250,1252,dl,decilitro,decilitro


In [ ]:
matches[matches['doc_id'] == 'S0210-48062004000500008-1']

In [ ]:
# patron = r'\(([A-Z]{2,8})\)'
# patron2 = r'[A-Z]{2,8}'
# patron3 = r'[a-z]+\-[A-Z]{1,8}'
# patron4 = r'[a-z]+\/[a-z]+'
# # create a list with them
# regexes = [ patron, patron2, patron3, patron4]
# for i in regexes:
#     generic_re = re.compile("(%s|%s|%s|%s)" % (patron, patron2, patron3, patron4))

In [ ]:
# texto = train_raw['texto'].iloc[3]
# #patron = re.compile(r'\((.*?)\)') #busca cosas dentro de dos paréntesis
# patron = re.compile(r'\(([A-Z]{2,8})\)') #entre 2 y 8 letras mayúsculas dentro de un paréntesis
# patron2 = re.compile(r'[A-Z]{2,8}')
# patron3 = re.compile(r'[a-z]+\-[A-Z]{1,8}')
# patron4 = re.compile(r'[a-z]+\/[a-z]+', re.I)

In [ ]:
# patron = re.compile(r'\(?([A-Z]{2,8})\)?') #entre 2 y 8 letras mayúsculas dentro de un paréntesis
# patron2 = re.compile(r'[A-Z]{2,8}')
# patron3 = re.compile(r'[a-z]+\-[A-Z]{1,8}')
# patron4 = re.compile(r'[a-z]+\/[a-z]+', re.I)

# # create a list with them
# regexes = [patron, patron2, patron3, patron4]

# # create the combined one
# pattern_combined = '|'.join(x.pattern for x in regexes)

# prueba = re.compile(pattern_combined)

In [ ]:
# abr = {}
# for i in range(0,train_raw.shape[0]):
#     texto = train_raw['texto'].iloc[i]
#     id_texto = train_raw['nombre'].iloc[i]
#     for i in regexes:
#         generic_re = re.compile("(%s|%s|%s)" % (patron2, patron3, patron4)).findall(texto)
#         for item in generic_re:
#             abr[id_texto] = item
# #abr

In [ ]:
# #set of abbreviations per text id, in the same line
# abr = {}
# for i in range(1,train_raw.shape[0]):
#     texto = train_raw['texto'].iloc[i]
#     id_texto = train_raw['nombre'].iloc[i]
#     li = patron.findall(texto)
#     for item in li:
#         abr[id_texto] = {item} | abr.get(id_texto) if abr.get(id_texto) else {item}

In [ ]:
# print(patron.findall(texto))
# print(patron2.findall(texto))
# print(patron3.findall(texto))
# print(patron.findall(texto))

In [ ]:
# <[a . . z ]>? \−? <[A . . Z]> ∗∗ { 1 . . 8 } <[a . . z\ \−\/]>? <[A . . Z0 .. 9 ] >+
# <[a . . z ]>∗

In [ ]:
# for i in range(10):
#     print(train_raw['texto'].iloc[i], "\n", re.findall(r'\((.*?)\)', train_raw['texto'].iloc[i]))

In [ ]:
# r = re.compile(r'\((.*?)\)', flags = re.I | re.I)
# print(train_raw['texto'].iloc[1], "\n", r.search(train_raw['texto'].iloc[1]).pos)

In [ ]:
# print(train_raw['texto'].iloc[1], "\n", re.findall(r'(\((.*?)\)|\b[A-Z]{3})', train_raw['texto'].iloc[1]))

In [ ]:
# od = collections.OrderedDict(sorted(abr.items()))
# len(od)